# h2o groupby queries

In [1]:
from pathlib import Path

import delta
import pyspark
import pyspark.sql.functions as F
from delta import configure_spark_with_delta_pip

In [2]:
builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.executor.memory", "10G")
    .config("spark.driver.memory", "25G")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
    .config("spark.sql.shuffle.partitions", "2")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/matthew.powers/opt/miniconda3/envs/pyspark-340-delta-240/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/matthew.powers/.ivy2/cache
The jars for the packages stored in: /Users/matthew.powers/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f79fb089-5d94-4411-9dc2-7f86ace3e5ce;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 104ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default    

In [3]:
df = spark.read.format("delta").load(f"{Path.home()}/data/deltalake/G1_1e9_1e2_0_0")

In [7]:
spark.sql("select * from {df} limit 3", df=df).show()

+-----+-----+------------+---+---+-------+---+---+---------+
|  id1|  id2|         id3|id4|id5|    id6| v1| v2|       v3|
+-----+-----+------------+---+---+-------+---+---+---------+
|id015|id010|id0000896846| 10|  1|5773276|  1|  1|33.170577|
|id017|id017|id0007133323| 71| 14|5476891|  3|  2|34.800229|
|id016|id030|id0002048672| 55| 35|4861331|  1|  3|32.064864|
+-----+-----+------------+---+---+-------+---+---+---------+



## Query 1

In [5]:
spark.sql("select id1, sum(v1) as v1 from {df} group by id1", df=df).show()

23/11/14 07:49:11 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----+--------+
|  id1|      v1|
+-----+--------+
|id016|30003304|
|id074|30006309|
|id070|29990210|
|id054|30011978|
|id053|29992360|
|id056|29987234|
|id057|29991822|
|id003|30003365|
|id001|30009448|
|id015|30006177|
|id017|29995061|
|id018|29992469|
|id014|29998476|
|id072|30003522|
|id071|29998357|
|id073|30006820|
|id055|30009993|
|id004|30015990|
|id002|29996534|
|id005|29993888|
+-----+--------+
only showing top 20 rows



In [6]:
spark.sql("select id1, sum(v1) as v1 from {df} group by id1", df=df).count()

100

## Query 2

In [8]:
spark.sql("select id1, id2, sum(v1) as v1 from {df} group by id1, id2", df=df).show()

+-----+-----+------+
|  id1|  id2|    v1|
+-----+-----+------+
|id016|id026|298873|
|id014|id009|299719|
|id017|id041|298918|
|id015|id100|299767|
|id016|id014|299652|
|id015|id018|299426|
|id014|id022|299753|
|id014|id044|300032|
|id015|id079|298299|
|id014|id025|300019|
|id014|id075|301299|
|id016|id079|300597|
|id017|id004|301294|
|id016|id061|299681|
|id015|id032|298705|
|id017|id036|298451|
|id017|id068|298285|
|id015|id072|300308|
|id017|id064|298988|
|id015|id085|301252|
+-----+-----+------+
only showing top 20 rows



In [9]:
spark.sql("select id1, id2, sum(v1) as v1 from {df} group by id1, id2", df=df).count()

10000

## Query 3

In [10]:
spark.sql("select id3, sum(v1) as v1, mean(v3) as v3 from {df} group by id3", df=df).show()

+------------+---+------------------+
|         id3| v1|                v3|
+------------+---+------------------+
|id0002654924|330|49.091885935185196|
|id0001975574|329|48.816218894230765|
|id0002470600|324|53.640876399999996|
|id0001698780|285|45.204360193548375|
|id0000058673|291|51.325111176470614|
|id0003895227|358| 49.58640469565216|
|id0005293826|295| 52.53704746315788|
|id0007581630|300|48.697405603960384|
|id0000651978|389| 49.91647398305082|
|id0003454065|274|51.122333543478256|
|id0001755020|342| 50.57228290434781|
|id0005409186|287| 53.28691733333334|
|id0005802677|310| 48.31412209708738|
|id0009759301|308|52.288704642857134|
|id0005675043|362| 47.65540899115044|
|id0008654924|258| 52.24759900000001|
|id0000248674|302|49.937610443298986|
|id0005200803|249| 43.05500375000001|
|id0000784417|289| 48.57986409183674|
|id0007793617|324|50.550142678899086|
+------------+---+------------------+
only showing top 20 rows



In [11]:
spark.sql("select id3, sum(v1) as v1, mean(v3) as v3 from {df} group by id3", df=df).count()

10000000

## Query 3 Modified

In [ ]:
%%time
spark.sql("select id3, sum(v1) as v1, mean(v3) as v3 from {df} group by id3 order by v1 desc limit 3", df=df).show()

## Query 4

In [12]:
spark.sql("select id4, mean(v1) as v1, mean(v2) as v2, mean(v3) as v3 from {df} group by id4", df=df).show()

+---+------------------+------------------+------------------+
|id4|                v1|                v2|                v3|
+---+------------------+------------------+------------------+
| 55|2.9998246661254955| 8.000023904618372|49.996973064034094|
| 29| 3.000572167759587| 7.999551868607876| 50.00342673344333|
| 82|2.9995379061458483| 7.999010613158845| 50.00021376928316|
| 87| 3.000169991381437| 8.000503174489053|50.011425763409626|
| 30|2.9985825421677204|7.9989537573132985| 50.00214866288746|
| 33|2.9999564146969644|7.9998203605744145| 50.00827978135219|
| 19| 2.999691681710651| 7.998959063016616| 49.99534693159616|
|  4| 2.999351427953455|  8.00024738933752| 50.00297873205454|
| 94| 2.999753562664744| 7.999774665861878| 50.00962994793641|
| 61| 3.000324935027996| 7.999546751139773|  49.9995310819004|
| 42|3.0005378493745725| 8.003788547788687| 50.00666532597359|
| 91| 3.000607368929785|  7.99814300074044| 50.01352919169919|
| 81| 2.999849930119025| 7.999589682350752| 49.99841869

In [13]:
spark.sql("select id4, mean(v1) as v1, mean(v2) as v2, mean(v3) as v3 from {df} group by id4", df=df).count()

100

## Query 5

In [14]:
spark.sql("select id6, sum(v1) as v1, sum(v2) as v2, sum(v3) as v3 from {df} group by id6", df=df).show()

+-------+---+---+------------------+
|    id6| v1| v2|                v3|
+-------+---+---+------------------+
|5773276|341|792| 5585.354910999999|
|2903102|356|973| 6205.401289000002|
|2466341|267|736| 4515.288610000001|
|2686046|281|769| 5316.997441000002|
|7454645|312|887| 4921.975717999999|
|6536815|335|916|       5886.131052|
|1079126|286|741|5324.4377319999985|
| 944877|282|817|5488.5830259999975|
|2693701|316|812|       5648.669043|
|3443772|334|965| 5950.841467999999|
|2328897|317|874| 5503.581474999999|
|1539817|222|646| 3475.920376999999|
| 566250|321|809| 4961.607664000001|
|8742933|244|703| 4397.223067000001|
|4081353|322|777| 4908.233546999998|
|2803042|263|749| 4180.598368999999|
| 982174|341|897| 6163.637482999999|
|5406874|299|795|       5459.568165|
|2323132|342|967| 5555.607695999996|
|7844955|298|739| 4931.196515999999|
+-------+---+---+------------------+
only showing top 20 rows



In [15]:
spark.sql("select id6, sum(v1) as v1, sum(v2) as v2, sum(v3) as v3 from {df} group by id6", df=df).count()

10000000

## Query 6

In [16]:
spark.sql("select id4, id5, median(v3) as median_v3, stddev(v3) as sd_v3 from {df} group by id4, id5", df=df).show()

+---+---+------------------+------------------+
|id4|id5|         median_v3|             sd_v3|
+---+---+------------------+------------------+
|  1|  1|        49.9667915|28.951202476533364|
|  1|  3|         49.973412| 28.75385061525536|
|  1|  4|50.044976500000004|28.871386653636858|
|  1|  8|        49.6763525|28.870302332845846|
|  1|  9|         49.967863| 28.85591006497422|
|  1| 12|         50.059923|28.878182404369404|
|  1| 13|50.030026500000005|28.913213008271043|
|  1| 16|         49.744494|28.841951330631197|
|  1| 19|         49.941295|28.846263544673924|
|  1| 21|49.767773000000005|28.869396596703453|
|  1| 24|         50.116194|28.881644688529512|
|  1| 25|         50.125431|28.880724070335713|
|  1| 27|          50.17253|28.893590675938803|
|  1| 28|         49.934926|28.858709263195134|
|  1| 31|49.900381499999995|28.850729085239575|
|  1| 32|          49.88918|28.866283940438468|
|  1| 33|         49.912413|28.792795893939132|
|  1| 35|         49.978273|28.822689995

In [17]:
spark.sql("select id4, id5, median(v3) as median_v3, stddev(v3) as sd_v3 from {df} group by id4, id5", df=df).count()

10000

## Query 7

In [18]:
spark.sql("select id3, max(v1)-min(v2) as range_v1_v2 from {df} group by id3", df=df).show()

+------------+-----------+
|         id3|range_v1_v2|
+------------+-----------+
|id0002654924|          4|
|id0001975574|          4|
|id0002470600|          4|
|id0001698780|          4|
|id0000058673|          4|
|id0003895227|          4|
|id0005293826|          4|
|id0007581630|          4|
|id0000651978|          4|
|id0003454065|          4|
|id0001755020|          4|
|id0005409186|          4|
|id0005802677|          4|
|id0009759301|          4|
|id0005675043|          4|
|id0008654924|          4|
|id0000248674|          4|
|id0005200803|          4|
|id0000784417|          4|
|id0007793617|          4|
+------------+-----------+
only showing top 20 rows



In [19]:
spark.sql("select id3, max(v1)-min(v2) as range_v1_v2 from {df} group by id3", df=df).count()

10000000

## Query 8

In [20]:
spark.sql("select id6, largest2_v3 from (select id6, v3 as largest2_v3, row_number() over (partition by id6 order by v3 desc) as order_v3 from {df} where v3 is not null) sub_query where order_v3 <= 2", df=df).show()

+---+-----------+
|id6|largest2_v3|
+---+-----------+
|  2|  98.933402|
|  2|  96.387642|
|  4|  97.930795|
|  4|  97.038094|
|  5|  97.234212|
|  5|  97.220159|
|  8|    99.6065|
|  8|  99.487874|
| 12|  99.699698|
| 12|  98.800794|
| 14|  99.586843|
| 14|  98.666442|
| 15|   99.38758|
| 15|    97.9149|
| 16|  99.827275|
| 16|  98.889332|
| 17|  96.784773|
| 17|  92.879215|
| 18|  97.062454|
| 18|  94.603703|
+---+-----------+
only showing top 20 rows



In [21]:
spark.sql("select id6, largest2_v3 from (select id6, v3 as largest2_v3, row_number() over (partition by id6 order by v3 desc) as order_v3 from {df} where v3 is not null) sub_query where order_v3 <= 2", df=df).count()

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 2) / 2]
Traceback (most recent call last):
  File "/Users/matthew.powers/opt/miniconda3/envs/pyspark-340-delta-240/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/matthew.powers/opt/miniconda3/envs/pyspark-340-delta-240/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/matthew.powers/opt/miniconda3/envs/pyspark-340-delta-240/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

## Query 9

In [ ]:
spark.sql("select id2, id4, pow(corr(v1, v2), 2) as r2 from {df} group by id2, id4", df=df).show()

In [ ]:
spark.sql("select id2, id4, pow(corr(v1, v2), 2) as r2 from {df} group by id2, id4", df=df).count()

## Query 10

In [ ]:
spark.sql("select id1, id2, id3, id4, id5, id6, sum(v3) as v3, count(*) as count from {df} group by id1, id2, id3, id4, id5, id6", df=df).show()

In [ ]:
spark.sql("select id1, id2, id3, id4, id5, id6, sum(v3) as v3, count(*) as count from {df} group by id1, id2, id3, id4, id5, id6", df=df).count()